In [1]:
import numpy as np
import pandas as pd


In [2]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')


In [3]:
train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']

test_x = test.copy()

In [4]:
#
# 特徴量の作成
#

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
train_x = train_x.drop(['PassengerId'], axis=1)
test_x = test_x.drop(['PassengerId'], axis=1)


In [7]:
train_x  = train_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x  = test_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [8]:
for c in ['Sex', 'Embarked']:
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))

In [9]:
#
# model
#

In [10]:
from xgboost import XGBClassifier

In [11]:
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=20, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=71, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [12]:
pred = model.predict_proba(test_x)[:, 1]


In [13]:
pred_label = np.where(pred > 0.5, 1, 0)

In [14]:
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submittion_first.csv', index=False)

In [15]:
#
# cross validation
#

In [16]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

In [17]:
scores_accuracy = []
scores_logloss = []

In [18]:
kf = KFold(n_splits=4, shuffle=True, random_state=71)

In [19]:
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x, tr_y)
    
    va_pred = model.predict_proba(va_x)[:, 1]
    
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

In [20]:
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)

In [21]:
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')


logloss: 0.4270, accuracy: 0.8148


In [22]:
#
# model tuning
#

In [23]:
import itertools

In [24]:
# tuning target parameters
param_space = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1.0, 2.0, 4.0]
}

In [25]:
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

In [26]:
params = []
scores = []

In [27]:
for max_depth, min_child_weight in param_combinations:
    score_folds = []
    
    # cross validation
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        model = XGBClassifier(n_estimators=20, random_state=71, max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)
        
        va_pred = model.predict_proba(va_x)[:, 1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
        
    score_mean = np.mean(score_folds)
    
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)
        

In [28]:
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]

In [29]:
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

max_depth: 7, min_child_weight: 2.0


In [30]:
#
#  ロジスティック回帰用の特徴量の作成
#

In [31]:
from sklearn.preprocessing import OneHotEncoder

In [32]:
train_x2 = train.drop(['Survived'], axis=1)
test_x2 = test.copy()


In [33]:
train_x2 = train_x2.drop(['PassengerId'], axis=1)
test_x2 = test_x2.drop(['PassengerId'], axis=1)

In [34]:
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [35]:
# one-hot encoding


In [36]:
cat_cols = ['Sex', 'Embarked', 'Pclass']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x2[cat_cols].fillna('NA'))

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=False)

In [37]:
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]

In [38]:
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns=ohe_columns)


In [39]:
train_x2 = train_x2.drop(cat_cols, axis=1)
test_x2 = test_x2.drop(cat_cols, axis=1)

train_x2 = pd.concat([train_x2, ohe_train_x2], axis=1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis=1)


In [40]:
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(), inplace=True)
    test_x2[col].fillna(train_x2[col].mean(), inplace=True)


In [41]:
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])

In [42]:
#
#  ensemble
#

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
model_xgb = XGBClassifier(n_estimators=20, random_state=71)
model_xgb.fit(train_x, train_y)

pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

In [45]:
model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
model_lr.fit(train_x2, train_y)

pred_lr = model_lr.predict_proba(test_x2)[:, 1]


In [46]:
print(len(train_x))
print(len(train_y))
print(len(train_x2))
print(len(pred_xgb))
print(len(pred_lr))


891
891
891
418
418


In [47]:
# 予測値の加重平均
pred = pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5, 1, 0)